In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from numpy import asarray
import glob
import cv2
import tensorflow as tf
from tensorflow import keras

#### Import Data

In [2]:
all_labels=pd.read_csv('train.csv')

In [4]:
Labels_DF=pd.DataFrame(all_labels)
Labels_DF.rename(columns={'image':'file_name'}, inplace=True)
Labels_DF.head(3)

,file_name,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b


In [9]:
Labels_DF['species'].value_counts()

bottlenose_dolphin           10781
beluga                        7443
humpback_whale                7392
blue_whale                    4830
false_killer_whale            3326
dusky_dolphin                 3139
killer_whale                  2455
spinner_dolphin               1700
melon_headed_whale            1689
minke_whale                   1608
fin_whale                     1324
gray_whale                    1123
southern_right_whale           866
spotted_dolphin                490
sei_whale                      428
short_finned_pilot_whale       367
common_dolphin                 347
cuviers_beaked_whale           341
pilot_whale                    262
long_finned_pilot_whale        238
white_sided_dolphin            229
brydes_whale                   154
pantropic_spotted_dolphin      145
globis                         116
commersons_dolphin              90
pygmy_killer_whale              76
rough_toothed_dolphin           60
frasiers_dolphin                14
Name: species, dtype

In [7]:
## species with incorrect spellings.  bottlenose_dolpin, kiler_whale.
Labels_DF['species'].replace('bottlenose_dolpin', 'bottlenose_dolphin', inplace=True)
Labels_DF['species'].replace('kiler_whale', 'killer_whale', inplace=True)

In [14]:
## create subspecies for whale or dolphin
sub=[]
for each in Labels_DF['species']:
    if 'dolphin' in each:
        a=1
    else: a=2
    sub.append(a)

In [15]:
Labels_DF['sub_cat']=sub

In [19]:
## create species cat numerical
from sklearn.preprocessing import OrdinalEncoder
OE=OrdinalEncoder()
SE=OE.fit_transform(Labels_DF[['species']])
Labels_DF['species_category']=SE.astype(int)

In [20]:
Labels_DF.head(6)

,file_name,species,individual_id,sub_cat,species_category
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,2,16
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,2,13
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,2,8
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,1,2
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,2,13
5,000a8f2d5c316a.jpg,bottlenose_dolphin,b9907151f66e,1,2


In [22]:
len(Labels_DF['species_category'].value_counts())

28

In [39]:
image_list = []
for filename in glob.glob('SmallSample/*.jpg'):
    im=Image.open(filename)
    image_list.append(im)

In [33]:
image_names = []
for filename in glob.glob('SmallSample/*.jpg'):
    name=filename.strip('SmallSample\\')
    image_names.append(name)

####  use image names to determine species label

In [35]:
Image_DF=pd.DataFrame({
    'image': image_list,
    'file_name': image_names
    })

In [36]:
Image_DF.head(3)

,image,file_name
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000a8f2d5c316a.jpg
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000be9acf46619.jpg
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000bef247c7a42.jpg


In [46]:
tf_files=[]
for each in image_list:
    ar=tf.keras.preprocessing.image.img_to_array(each)
    tf_files.append(ar)


In [47]:
Image_DF['tf_images']=tf_files

In [48]:
Image_DF.head(4)

,image,file_name,tf_images
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000a8f2d5c316a.jpg,"[[[162.0, 163.0, 165.0], [163.0, 164.0, 166.0]..."
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000be9acf46619.jpg,"[[[80.0, 95.0, 102.0], [80.0, 95.0, 102.0], [8..."
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000bef247c7a42.jpg,"[[[153.0, 195.0, 235.0], [153.0, 195.0, 235.0]..."
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000c3d63069748.jpg,"[[[101.0, 132.0, 163.0], [101.0, 132.0, 163.0]..."


In [49]:
## Join Dataframes

Big_DF=Image_DF.merge(Labels_DF, how='outer')

In [50]:
Big_DF.head(6)

,image,file_name,tf_images,species,individual_id,sub_cat,species_category
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000a8f2d5c316a.jpg,"[[[162.0, 163.0, 165.0], [163.0, 164.0, 166.0]...",bottlenose_dolphin,b9907151f66e,1,2
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000be9acf46619.jpg,"[[[80.0, 95.0, 102.0], [80.0, 95.0, 102.0], [8...",beluga,afb9b3978217,2,0
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000bef247c7a42.jpg,"[[[153.0, 195.0, 235.0], [153.0, 195.0, 235.0]...",humpback_whale,444d8894ccc8,2,13
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000c3d63069748.jpg,"[[[101.0, 132.0, 163.0], [101.0, 132.0, 163.0]...",beluga,df94b15285b9,2,0
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,000c476c11bad5.jpg,"[[[36.0, 62.0, 79.0], [36.0, 62.0, 79.0], [37....",bottlenose_dolphin,b11b2404c7e3,1,2
5,<PIL.JpegImagePlugin.JpegImageFile image mode=...,00a198105cdc24.jpg,"[[[46.0, 86.0, 145.0], [39.0, 79.0, 138.0], [3...",humpback_whale,da611bc6c389,2,13


In [55]:
Big_DF['tf_images'][0].shape

(2304, 3456, 3)